In [1]:
import os
import pickle

from test import test
from train import train
from models import train_model_all_string, test_model
from models import get_next_tokens, get_next_n_tokens
from token_id_generator import TokenIdGenerator
from tokenizer import tokenize_java_code
from tree_node import TreeNode
from token_id_generator import TokenIdGenerator

**directory_train**, **directory_test** - пути к директориям с проетами JAVA разделенные на тренировочные и тестовые</br>
**max_file_count** - ограничение на количество обучаемых файлов кода (нужно в связи с ограничением по памяти и времени) </br>
**max_tokens** - максимальное количество подряд идущих токенов которые будут записаны в дерево 

In [61]:
directory_train = os.getenv("PATH_TO_JAVA_CORPUS_TRAIN")
directory_test = os.getenv("PATH_TO_JAVA_CORPUS_TEST")
max_file_count = 2000
max_tokens = 10

пути для записи и чтения файла модели генератора id

In [62]:
model_path = f"trained_models/tree{max_file_count}_{max_tokens}.pkl"
id_generator_path = f"trained_models/id{max_file_count}_{max_tokens}.pkl"

Пример тренировки модели

In [4]:
result = train(
    directory_train, 
    train_model_all_string, 
    max_file_count=max_file_count,
    max_tokens=max_tokens
)

processed files: 2008


In [64]:
root_node = result["root_node"]
id_generator = result["id_generator"]

In [65]:
root_node.dump(model_path)
id_generator.dump(id_generator_path)

In [66]:
root_node = TreeNode("").load(model_path)
id_generator = TokenIdGenerator().load(id_generator_path)

Примеры тестирования модели

In [8]:
test(directory_test,
     root_node, 
     id_generator, 
     test_model, 
     max_file_count=10, 
     max_tokens=max_tokens
)

processed files 13: 42.07%


{'processed_files_count': 13,
 'percent': 0.42065329218106995,
 'count_true': 3271,
 'count_false': 4505,
 'count_true_whitespace_or_newline': 3483}

In [9]:
test(directory_train,
     root_node, 
     id_generator, 
     test_model, 
     max_file_count=200, 
     max_tokens=max_tokens
)

processed files 201: 64.12%


{'processed_files_count': 201,
 'percent': 0.6411824312519655,
 'count_true': 55050,
 'count_false': 30807,
 'count_true_whitespace_or_newline': 41871}

Пример работы

In [101]:
test = '''import java.'''

tokens = tokenize_java_code(test)

node = root_node
returned_tokens = get_next_n_tokens(
    node, max_tokens, 
    id_generator, 
    tokens, 
    count_reterned_tokens = 5, 
    n = 1
)

results = [[test] + [x[1] for x in returned_token] for returned_token in returned_tokens]
for result in results:
    for x in result:
        print([x], end='')
    print("\n"+"-"*35)

['import java.']['util']
-----------------------------------
['import java.']['io']
-----------------------------------
['import java.']['awt']
-----------------------------------
['import java.']['net']
-----------------------------------
['import java.']['lang']
-----------------------------------
